In [1]:
import tweepy
from tweepy import OAuthHandler
from tweepy import API

import time
from tweepy import Stream
from dateutil import parser
import json

# Jack Nugent Keys
C_KEY = 'qDLzDR1kgKZrwc0nfCiknCNak'
C_SECRET = '0siNqGf4CSQVM2EL2JTYKq0fqBN7VXtTXv2MSyUAiiX5V1DBeG'
A_TOKEN_KEY = '762757536827121664-YNdvEO2FsulfLcjUJcezb5OMRx3Nlgs'
A_TOKEN_SECRET = '88DYUdSwCfWmNlBWbY9It8uFqajxE7NMMRS4moZbdjz4k'
BEARER = "AAAAAAAAAAAAAAAAAAAAAOCnhQEAAAAABJxPmDWLGJXfFdBa9Nprs%2BfFHus%3De6Qt9gjlzObpDy4HIWJWMQUXXLE6nkNUF3Drkv6QLkZezBn4JU"

In [2]:
FILE = 'output_geotag_day2.json'

class MyStreamingClient(tweepy.StreamingClient):
    def __init__(self, bearer_token, time_limit=7200):
        self.start_time = time.time()
        self.limit = time_limit
        self.outFile = open(FILE, 'w')
        super(MyStreamingClient, self).__init__(bearer_token)
        
#     def on_data(self, data_item):
        
#         ### Modify on_data
#         dict_data = {}
#         try:
#             data = json.loads(data_item)
#             if data['coordinates'] or data['geo']:
#                 print("COORDS:",data['coordinates'], '\nGEO:', data['geo'])
#                 dict_data['created_at'] = parser.parse(data['created_at'], ignoretz=True)
#                 dict_data['text'] = data['text']
#                 dict_data['id_str'] = data['id_str']
#                 dict_data['user_id'] = data['user']['id_str']
#                 dict_data['user_name'] = data['user']['id_str']
#                 dict_data['lat'] = str(data['coordinates']['coordinates'][1])
#                 dict_data['lon'] = str(data['coordinates']['coordinates'][0])
#                 dict_data['language'] = data['user']['lang']
                
#             print(dict_data)
#             self.outFile.write(json.dumps(dict_data))
#             self.outFile.write('\n')
#         except:
# #             print('FAIL')
#             pass
#         ###

#         if (time.time() - self.start_time) < self.limit:
#             return True
#         else:
#             self.outFile.close()
#             self.disconnect()
#             return False

    def on_data(self, data):
        if (time.time() - self.start_time) < self.limit:
            self.outFile.write(data.decode("utf-8"))
            self.outFile.write('\n')
            return True
        else:
            self.outFile.close()
            self.disconnect()
            return False
        
    def on_error(self, status):
        print(status)

# Function to remove all rules from an existing stream_client
def remove_rules(stream_client):
    rules = list(stream_client.get_rules())[0]
    if rules != None:
        for rule in rules:
            id_ = rule.id
            stream_client.delete_rules(id_)
        
s_c = MyStreamingClient(bearer_token=BEARER)
remove_rules(s_c)

In [4]:
# All Rule
s_c.add_rules(tweepy.StreamRule('(Musk OR Elon) -is:retweet lang:en has:geo place_country:US'))

s_c.get_rules()

Response(data=[StreamRule(value='(Musk OR Elon) -is:retweet lang:en has:geo place_country:US', tag=None, id='1595482959607263232')], includes={}, errors=[], meta={'sent': '2022-11-23T18:22:35.008Z', 'result_count': 1})

In [5]:
import datetime as dt
print(dt.datetime.now())
s_c.filter(tweet_fields=["entities", 'geo', 'created_at'], 
           place_fields=["id","geo","name","country_code","place_type","full_name","country"],
          expansions=['geo.place_id'])
print(dt.datetime.now())

2022-11-23 13:22:35.065030


Stream connection closed by Twitter


2022-11-23 15:22:42.523561


---

In [10]:
data_json = []

In [11]:
f = open('output_geotag_day1.json')

for line in f:
    try:
        data_json.append(json.loads(line))
    # Discard Tweets with invalid chars
    except:
        pass
    
f.close()

In [12]:
geo_count = 0
for tweet in data_json:
    if len(tweet['data']['geo']) > 0:
        geo_count += 1
#         print(tweet['data'])
#         print()
        
print("{} Tweets out of {} ({:.2%})".format(geo_count, len(data_json), geo_count/len(data_json)))

312 Tweets out of 312 (100.00%)
